# 전처리된 review 데이터 불러오기

In [ ]:
import csv
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
from tqdm.auto import trange
import numpy as np
import os

def get_data(path = "/content/drive/MyDrive/학교/졸업작품/"):
    input_file = path + "reviews.txt"
    data = []
    with open(input_file, 'r', newline='', encoding='utf-8') as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            data.append(row)
    return data

In [ ]:
data=get_data()

In [ ]:
data_df = pd.DataFrame(data)

In [ ]:
data_df = data_df.rename(columns={3:'text'})

In [ ]:
data_df

,0,1,2,text
0,0,5758,5.0,First time visit and not disappointed. From th...
1,0,5275,5.0,"I had a recent move from West Chester, PA to K..."
2,1,2410,5.0,I'm a vegan and my father in law picked me up ...
3,1,8714,4.0,We've been to Horizons several times now. My h...
4,2,8586,2.0,I was looking for a place in Old City to get b...
...,...,...,...,...
968117,279961,778,1.0,Avis car rental should not be in business. We...
968118,279962,1979,5.0,"Nice atmosphere, food and drinks are perfect, ..."
968119,279962,10210,5.0,"Haven been to this spot twice, both times were..."
968120,279963,4308,5.0,Absolutely some of the best ramen I've had - i...


# BERT 모델 불러오기

In [ ]:
# Pretrained 모델의 이름
# https://huggingface.co/textattack/bert-base-uncased-yelp-polarity
model_name = "textattack/bert-base-uncased-yelp-polarity"

# AutoTokenizer와 AutoModel 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## [CLS] 토큰 방식

In [ ]:
# import torch
# text = "Here is some text to encode"

# # 텍스트를 모델 입력 형식으로 토큰화
# inputs = tokenizer(text, return_tensors="pt")

# # 모델을 실행하여 출력값 추출
# with torch.no_grad():  # 그라디언트 계산 비활성화
#     outputs = model(**inputs)

# # 모델의 마지막 히든 스테이트를 추출 (각 토큰의 임베딩)
# last_hidden_states = outputs.last_hidden_state

# # 문장 전체에 대한 임베딩을 원한다면 [CLS] 토큰의 임베딩을 사용
# cls_embedding = last_hidden_states[:, 0, :]


In [ ]:
# cls_embedding.shape

torch.Size([1, 768])

## 전체 토큰 평균 방식

In [ ]:
# # 분석할 텍스트
# text = "Here is some text to encode"

# # 텍스트를 모델이 이해할 수 있는 형태로 토큰화
# inputs = tokenizer(text, return_tensors="pt")

# # 모델 실행하여 출력 추출
# with torch.no_grad():  # 그라디언트 계산 비활성화
#     outputs = model(**inputs)

# # 모델의 마지막 히든 스테이트를 추출
# last_hidden_states = outputs.last_hidden_state

# # 토큰 임베딩들의 평균 계산
# sentence_embedding = last_hidden_states.mean(dim=1)

In [ ]:
# sentence_embedding.shape

torch.Size([1, 768])

# 리뷰 to 임베딩
전체 리뷰 임베딩 생성

In [ ]:
# GPU 사용 가능 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 모델을 GPU로 이동
model = model.to(device)

Using device: cuda


In [ ]:
def get_bert_embeddings(texts):
    # 텍스트를 토큰화하고 배치로 처리
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=256).to(device)
    # 모델 실행하여 출력 추출
    with torch.no_grad():
        outputs = model(**inputs)
    # 마지막 히든 스테이트 추출 및 평균 임베딩 계산
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.cpu()  # GPU에서 CPU로 데이터 이동

첫 시도:

In [ ]:
batch_size = 1000
all_embeddings = None  # 전체 임베딩을 저장할 변수
output_dir = '/content/drive/MyDrive/학교/졸업작품/'
temp_file_template = os.path.join(output_dir, 'temp_embeddings.npy')  # 임시 파일 경로
final_output_file = os.path.join(output_dir, 'embeddings.npy')  # 최종 파일 경로

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for i in trange(0, len(data_df), batch_size):
    batch_texts = data_df['text'][i:i + batch_size].tolist()
    batch_embeddings = get_bert_embeddings(batch_texts)
    # 임베딩을 numpy 배열로 변환
    embeddings_array = batch_embeddings.numpy()
    # 이전의 모든 임베딩과 현재 배치의 임베딩을 결합
    if all_embeddings is None:
        all_embeddings = embeddings_array
    else:
        all_embeddings = np.concatenate((all_embeddings, embeddings_array), axis=0)

    # i가 10,000의 배수일 때 임시 파일로 저장
    if (i + batch_size) % 10000 == 0 or (i + batch_size) >= len(data_df):
        np.save(temp_file_template, all_embeddings)
        print(f"임시 저장 완료: {temp_file_template} at index {i}")

# 최종적으로 모든 임베딩을 하나의 npy 파일에 저장
np.save(final_output_file, all_embeddings)
print("모든 배치 처리 완료, 최종 파일 저장됨.")

  0%|          | 0/969 [00:00<?, ?it/s]

임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 9000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 19000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 29000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 39000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 49000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 59000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 69000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 79000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 89000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 99000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 109000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 119000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 129000
임시 저장 완료: 

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.93 GiB. GPU 0 has a total capacity of 15.77 GiB of which 1.76 GiB is free. Process 172298 has 14.01 GiB memory in use. Of the allocated memory 13.61 GiB is allocated by PyTorch, and 40.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

중단될시 세이브 포인트에서 시작:

In [ ]:
# 임시 파일에서 저장된 데이터를 불러옵니다.
all_embeddings = np.load('/content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy')

# 중단된 index 다음부터 시작
start_index = 570000  # 569,000 다음 index
batch_size = 1000
output_dir = '/content/drive/MyDrive/학교/졸업작품/'
temp_file_template = os.path.join(output_dir, 'temp_embeddings.npy')
final_output_file = os.path.join(output_dir, 'embeddings.npy')

for i in trange(start_index, len(data_df), batch_size):
    batch_texts = data_df['text'][i:i + batch_size].tolist()
    batch_embeddings = get_bert_embeddings(batch_texts)
    # 임베딩을 numpy 배열로 변환
    embeddings_array = batch_embeddings.numpy()
    # 이전의 모든 임베딩과 현재 배치의 임베딩을 결합
    all_embeddings = np.concatenate((all_embeddings, embeddings_array), axis=0)

    # i가 10,000의 배수일 때 임시 파일로 저장
    if (i + batch_size) % 10000 == 0 or (i + batch_size) >= len(data_df):
        np.save(temp_file_template, all_embeddings)
        print(f"임시 저장 완료: {temp_file_template} at index {i}")

# 최종적으로 모든 임베딩을 하나의 npy 파일에 저장
np.save(final_output_file, all_embeddings)
print("모든 배치 처리 완료, 최종 파일 저장됨.")

  0%|          | 0/399 [00:00<?, ?it/s]

임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 579000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 589000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 599000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 609000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 619000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 629000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 639000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 649000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 659000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 669000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 679000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 689000
임시 저장 완료: /content/drive/MyDrive/학교/졸업작품/temp_embeddings.npy at index 699000

In [ ]:
data = np.load('/content/drive/MyDrive/학교/졸업작품/embeddings.npy')

In [ ]:
data.shape

(968122, 768)